# 📸 스테레오비전 기초 실습

이 노트북에서는 OpenCV를 사용하여 기본적인 스테레오비전을 구현하고, StereoBM과 StereoSGBM 알고리즘을 비교해보겠습니다.

## 🎯 학습 목표
- OpenCV의 스테레오비전 기본 사용법 익히기
- StereoBM과 StereoSGBM 알고리즘 비교
- 매개변수 조정이 결과에 미치는 영향 관찰
- 시차 맵(Disparity Map) 생성 및 분석

## 📦 필요한 라이브러리 import

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import os
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (15, 10)

print("✅ 라이브러리 import 완료!")
print(f"OpenCV 버전: {cv2.__version__}")

## 🖼️ 테스트 이미지 생성

실제 스테레오 이미지가 없는 경우를 위해 테스트용 이미지를 생성합니다.

In [ ]:
def create_test_stereo_images():
    """
    테스트용 스테레오 이미지 쌍 생성
    """
    height, width = 480, 640
    
    # 좌측 이미지 생성
    left_img = np.zeros((height, width), dtype=np.uint8)
    
    # 다양한 패턴 추가
    # 사각형들 (서로 다른 깊이)
    cv2.rectangle(left_img, (100, 100), (200, 200), 255, -1)  # 흰색 사각형
    cv2.rectangle(left_img, (300, 150), (400, 250), 180, -1)  # 회색 사각형
    cv2.rectangle(left_img, (150, 300), (250, 400), 120, -1)  # 어두운 회색 사각형
    
    # 원들
    cv2.circle(left_img, (500, 100), 50, 200, -1)
    cv2.circle(left_img, (450, 350), 30, 150, -1)
    
    # 선들
    cv2.line(left_img, (50, 50), (590, 50), 100, 3)
    cv2.line(left_img, (50, 430), (590, 430), 100, 3)
    
    # 우측 이미지 생성 (좌측에서 약간 이동)
    right_img = np.zeros((height, width), dtype=np.uint8)
    
    # 동일한 패턴을 약간 이동시켜 생성 (시차 효과)
    cv2.rectangle(right_img, (85, 100), (185, 200), 255, -1)   # 15픽셀 왼쪽 이동
    cv2.rectangle(right_img, (280, 150), (380, 250), 180, -1)  # 20픽셀 왼쪽 이동
    cv2.rectangle(right_img, (140, 300), (240, 400), 120, -1)  # 10픽셀 왼쪽 이동
    
    cv2.circle(right_img, (475, 100), 50, 200, -1)  # 25픽셀 왼쪽 이동
    cv2.circle(right_img, (445, 350), 30, 150, -1)  # 5픽셀 왼쪽 이동
    
    cv2.line(right_img, (50, 50), (590, 50), 100, 3)
    cv2.line(right_img, (50, 430), (590, 430), 100, 3)
    
    # 노이즈 추가로 더 현실적으로 만들기
    noise_left = np.random.randint(0, 30, (height, width))
    noise_right = np.random.randint(0, 30, (height, width))
    
    left_img = np.clip(left_img.astype(np.int16) + noise_left, 0, 255).astype(np.uint8)
    right_img = np.clip(right_img.astype(np.int16) + noise_right, 0, 255).astype(np.uint8)
    
    return left_img, right_img

# 테스트 이미지 생성
left_image, right_image = create_test_stereo_images()

print("✅ 테스트 스테레오 이미지 생성 완료!")
print(f"이미지 크기: {left_image.shape}")

## 👁️ 생성된 스테레오 이미지 확인

In [ ]:
# 스테레오 이미지 시각화
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Left Image (좌측 이미지)', fontsize=14)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Right Image (우측 이미지)', fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

print("👀 좌우 이미지를 비교해보세요!")
print("💡 물체들이 우측 이미지에서 약간 왼쪽으로 이동한 것을 확인할 수 있습니다.")
print("   이것이 바로 시차(Disparity)입니다!")

## 🔧 StereoBM 알고리즘 구현

StereoBM(Stereo Block Matching)은 OpenCV에서 제공하는 빠른 스테레오 매칭 알고리즘입니다.

In [ ]:
def create_stereo_bm(num_disparities=64, block_size=15):
    """
    StereoBM 객체 생성
    
    Parameters:
    - num_disparities: 시차 범위 (16의 배수여야 함)
    - block_size: 블록 크기 (홀수여야 함)
    """
    # StereoBM 객체 생성
    stereo_bm = cv2.StereoBM_create(numDisparities=num_disparities, 
                                   blockSize=block_size)
    
    # 추가 매개변수 설정
    stereo_bm.setPreFilterCap(31)        # 전처리 필터 캡
    stereo_bm.setMinDisparity(0)         # 최소 시차
    stereo_bm.setTextureThreshold(10)    # 텍스처 임계값
    stereo_bm.setUniquenessRatio(10)     # 유일성 비율
    stereo_bm.setSpeckleWindowSize(100)  # 스펙클 윈도우 크기
    stereo_bm.setSpeckleRange(32)        # 스펙클 범위
    
    return stereo_bm

# StereoBM 객체 생성
stereo_bm = create_stereo_bm(num_disparities=64, block_size=15)

print("✅ StereoBM 객체 생성 완료!")
print(f"📊 설정된 매개변수:")
print(f"   - 시차 범위: {stereo_bm.getNumDisparities()}")
print(f"   - 블록 크기: {stereo_bm.getBlockSize()}")
print(f"   - 최소 시차: {stereo_bm.getMinDisparity()}")

## 🔬 StereoBM으로 시차 맵 계산

In [ ]:
# 시차 맵 계산
print("🔄 StereoBM으로 시차 맵 계산 중...")
disparity_bm = stereo_bm.compute(left_image, right_image)

# 시차 맵 정규화 (시각화를 위해)
disparity_bm_normalized = cv2.normalize(disparity_bm, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

print("✅ StereoBM 시차 맵 계산 완료!")
print(f"📊 시차 맵 통계:")
print(f"   - 최소값: {disparity_bm.min()}")
print(f"   - 최대값: {disparity_bm.max()}")
print(f"   - 평균값: {disparity_bm.mean():.2f}")
print(f"   - 표준편차: {disparity_bm.std():.2f}")

## 🎨 StereoBM 결과 시각화

In [ ]:
plt.figure(figsize=(18, 6))

# 좌측 이미지
plt.subplot(1, 3, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Left Image', fontsize=14)
plt.axis('off')

# 우측 이미지
plt.subplot(1, 3, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Right Image', fontsize=14)
plt.axis('off')

# StereoBM 시차 맵
plt.subplot(1, 3, 3)
plt.imshow(disparity_bm_normalized, cmap='plasma')
plt.title('StereoBM Disparity Map', fontsize=14)
plt.colorbar(label='Disparity (pixels)')
plt.axis('off')

plt.tight_layout()
plt.show()

print("🎨 시차 맵 해석:")
print("   - 밝은 부분 (노란색/흰색): 가까운 물체 (큰 시차)")
print("   - 어두운 부분 (보라색/검은색): 먼 물체 (작은 시차)")
print("   - 색상 변화: 거리의 변화를 나타냄")

## 🚀 StereoSGBM 알고리즘 구현

StereoSGBM(Semi-Global Block Matching)은 더 정확하지만 느린 알고리즘입니다.

In [ ]:
def create_stereo_sgbm(num_disparities=64, block_size=5):
    """
    StereoSGBM 객체 생성
    
    Parameters:
    - num_disparities: 시차 범위 (16의 배수)
    - block_size: 블록 크기
    """
    stereo_sgbm = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities=num_disparities,
        blockSize=block_size,
        P1=8 * 3 * block_size**2,      # 작은 시차 변화에 대한 패널티
        P2=32 * 3 * block_size**2,     # 큰 시차 변화에 대한 패널티
        disp12MaxDiff=1,               # 좌우 일관성 검사
        uniquenessRatio=10,            # 유일성 비율
        speckleWindowSize=100,         # 스펙클 윈도우 크기
        speckleRange=32                # 스펙클 범위
    )
    
    return stereo_sgbm

# StereoSGBM 객체 생성
stereo_sgbm = create_stereo_sgbm(num_disparities=64, block_size=5)

print("✅ StereoSGBM 객체 생성 완료!")
print(f"📊 설정된 매개변수:")
print(f"   - 시차 범위: {stereo_sgbm.getNumDisparities()}")
print(f"   - 블록 크기: {stereo_sgbm.getBlockSize()}")
print(f"   - P1 (작은 변화 패널티): {stereo_sgbm.getP1()}")
print(f"   - P2 (큰 변화 패널티): {stereo_sgbm.getP2()}")

## 🔬 StereoSGBM으로 시차 맵 계산

In [ ]:
# 시차 맵 계산
print("🔄 StereoSGBM으로 시차 맵 계산 중...")
disparity_sgbm = stereo_sgbm.compute(left_image, right_image)

# StereoSGBM은 16배수로 반환하므로 실제 시차값으로 변환
disparity_sgbm_real = disparity_sgbm.astype(np.float32) / 16.0

# 시각화를 위한 정규화
disparity_sgbm_normalized = cv2.normalize(disparity_sgbm_real, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

print("✅ StereoSGBM 시차 맵 계산 완료!")
print(f"📊 시차 맵 통계:")
print(f"   - 최소값: {disparity_sgbm_real.min():.2f}")
print(f"   - 최대값: {disparity_sgbm_real.max():.2f}")
print(f"   - 평균값: {disparity_sgbm_real.mean():.2f}")
print(f"   - 표준편차: {disparity_sgbm_real.std():.2f}")

## 🔄 StereoBM vs StereoSGBM 비교

In [ ]:
plt.figure(figsize=(20, 12))

# 원본 이미지들
plt.subplot(2, 3, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Left Image', fontsize=16)
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Right Image', fontsize=16)
plt.axis('off')

# 공간 비우기
plt.subplot(2, 3, 3)
plt.axis('off')

# 시차 맵 비교
plt.subplot(2, 3, 4)
plt.imshow(disparity_bm_normalized, cmap='plasma')
plt.title('StereoBM Result\n(빠름, 덜 정확)', fontsize=16)
plt.colorbar(label='Disparity')
plt.axis('off')

plt.subplot(2, 3, 5)
plt.imshow(disparity_sgbm_normalized, cmap='plasma')
plt.title('StereoSGBM Result\n(느림, 더 정확)', fontsize=16)
plt.colorbar(label='Disparity')
plt.axis('off')

# 차이 맵
plt.subplot(2, 3, 6)
# 크기를 맞춰서 차이 계산
diff = np.abs(disparity_bm_normalized.astype(np.float32) - disparity_sgbm_normalized.astype(np.float32))
plt.imshow(diff, cmap='hot')
plt.title('Difference Map\n(BM - SGBM)', fontsize=16)
plt.colorbar(label='Difference')
plt.axis('off')

plt.tight_layout()
plt.show()

print("🔍 알고리즘 비교 분석:")
print("   📏 StereoBM: 계산 속도가 빠르지만 노이즈가 많음")
print("   🎯 StereoSGBM: 계산 속도가 느리지만 더 정확하고 부드러움")
print("   🌈 Difference Map: 두 알고리즘의 차이를 보여줌 (빨간색일수록 차이가 큼)")

## 🎛️ 매개변수 조정 실험

다양한 매개변수 설정이 결과에 미치는 영향을 확인해보겠습니다.

In [ ]:
# 다양한 매개변수로 실험
parameter_sets = [
    {'num_disparities': 32, 'block_size': 5, 'name': 'Low Disparity, Small Block'},
    {'num_disparities': 64, 'block_size': 15, 'name': 'Medium Disparity, Large Block'},
    {'num_disparities': 96, 'block_size': 9, 'name': 'High Disparity, Medium Block'}
]

results = []

for params in parameter_sets:
    print(f"🔄 {params['name']} 실험 중...")
    
    # StereoBM 생성 및 계산
    stereo_test = create_stereo_bm(params['num_disparities'], params['block_size'])
    disparity_test = stereo_test.compute(left_image, right_image)
    disparity_test_norm = cv2.normalize(disparity_test, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    results.append({
        'disparity': disparity_test_norm,
        'params': params,
        'stats': {
            'min': disparity_test.min(),
            'max': disparity_test.max(),
            'mean': disparity_test.mean(),
            'std': disparity_test.std()
        }
    })

print("✅ 매개변수 실험 완료!")

## 📊 매개변수 실험 결과 시각화

In [ ]:
plt.figure(figsize=(18, 6))

for i, result in enumerate(results):
    plt.subplot(1, 3, i+1)
    plt.imshow(result['disparity'], cmap='plasma')
    plt.title(f"{result['params']['name']}\n"
             f"Disp: {result['params']['num_disparities']}, "
             f"Block: {result['params']['block_size']}", fontsize=12)
    plt.colorbar(label='Disparity')
    plt.axis('off')

plt.tight_layout()
plt.show()

# 통계 비교
print("📊 매개변수별 통계 비교:")
print("-" * 80)
print(f"{'Parameter Set':<30} {'Min':<8} {'Max':<8} {'Mean':<8} {'Std':<8}")
print("-" * 80)

for result in results:
    stats = result['stats']
    name = result['params']['name']
    print(f"{name:<30} {stats['min']:<8.1f} {stats['max']:<8.1f} {stats['mean']:<8.1f} {stats['std']:<8.1f}")

print("\n💡 매개변수 조정 팁:")
print("   🔢 numDisparities: 클수록 더 먼 물체까지 감지, 하지만 계산 시간 증가")
print("   🔳 blockSize: 클수록 노이즈 감소하지만 경계가 흐려짐")
print("   ⚖️ 속도 vs 품질의 트레이드오프를 고려하여 선택")

## 🧮 실제 거리 계산

시차 맵을 실제 거리 정보로 변환해보겠습니다.

In [ ]:
def disparity_to_depth(disparity, focal_length, baseline):
    """
    시차를 실제 거리로 변환
    공식: depth = (focal_length * baseline) / disparity
    
    Parameters:
    - disparity: 시차 맵
    - focal_length: 초점거리 (픽셀)
    - baseline: 베이스라인 (미터)
    """
    # 0으로 나누기 방지
    safe_disparity = np.where(disparity > 0, disparity, 1)
    
    # 거리 계산
    depth = (focal_length * baseline) / safe_disparity
    
    # 시차가 0인 곳은 무한대로 설정
    depth = np.where(disparity > 0, depth, np.inf)
    
    return depth

# 카메라 매개변수 설정 (예시)
focal_length = 500  # 픽셀 단위
baseline = 0.1      # 미터 단위 (10cm)

# 깊이 맵 계산
depth_map = disparity_to_depth(disparity_sgbm_real, focal_length, baseline)

# 너무 먼 거리는 제한 (시각화를 위해)
depth_map_clipped = np.clip(depth_map, 0, 10)  # 10미터로 제한

print("✅ 깊이 맵 계산 완료!")
print(f"📊 거리 통계 (유효한 픽셀만):")
valid_depths = depth_map_clipped[depth_map_clipped < 10]
if len(valid_depths) > 0:
    print(f"   - 최소 거리: {valid_depths.min():.2f} m")
    print(f"   - 최대 거리: {valid_depths.max():.2f} m")
    print(f"   - 평균 거리: {valid_depths.mean():.2f} m")
    print(f"   - 표준편차: {valid_depths.std():.2f} m")

## 🌈 시차 맵과 깊이 맵 비교

In [ ]:
plt.figure(figsize=(18, 6))

# 원본 이미지
plt.subplot(1, 3, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Original Left Image', fontsize=14)
plt.axis('off')

# 시차 맵
plt.subplot(1, 3, 2)
plt.imshow(disparity_sgbm_normalized, cmap='plasma')
plt.title('Disparity Map\n(픽셀 단위)', fontsize=14)
plt.colorbar(label='Disparity (pixels)')
plt.axis('off')

# 깊이 맵
plt.subplot(1, 3, 3)
plt.imshow(depth_map_clipped, cmap='viridis')
plt.title('Depth Map\n(실제 거리)', fontsize=14)
plt.colorbar(label='Distance (meters)')
plt.axis('off')

plt.tight_layout()
plt.show()

print("🎨 깊이 맵 해석:")
print("   - 어두운 부분 (보라색): 가까운 물체")
print("   - 밝은 부분 (노란색): 먼 물체")
print("   - 색상 변화: 실제 거리의 변화를 미터 단위로 표현")
print("\n🔬 시차 vs 깊이의 관계:")
print("   - 시차가 클수록 → 거리가 가까움")
print("   - 시차가 작을수록 → 거리가 멀음")
print("   - 역비례 관계: depth = (f × b) / disparity")

## 📈 시차-거리 관계 그래프

In [ ]:
# 이론적 시차-거리 관계 그래프
disparity_range = np.arange(1, 65, 1)  # 1~64 픽셀
theoretical_depth = (focal_length * baseline) / disparity_range

plt.figure(figsize=(12, 8))

# 이론적 관계
plt.subplot(2, 1, 1)
plt.plot(disparity_range, theoretical_depth, 'b-', linewidth=2, label='Theoretical')
plt.xlabel('Disparity (pixels)')
plt.ylabel('Distance (meters)')
plt.title('Theoretical Disparity-Distance Relationship\n(이론적 시차-거리 관계)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(0, 65)
plt.ylim(0, 5)

# 실제 데이터 분포
plt.subplot(2, 1, 2)
# 유효한 데이터만 선택
valid_mask = (disparity_sgbm_real > 0) & (depth_map_clipped < 10)
valid_disparity = disparity_sgbm_real[valid_mask]
valid_depth = depth_map_clipped[valid_mask]

if len(valid_disparity) > 0:
    # 너무 많은 점은 샘플링
    if len(valid_disparity) > 5000:
        indices = np.random.choice(len(valid_disparity), 5000, replace=False)
        valid_disparity = valid_disparity[indices]
        valid_depth = valid_depth[indices]
    
    plt.scatter(valid_disparity, valid_depth, alpha=0.5, s=1, c='red', label='Actual Data')
    plt.plot(disparity_range, theoretical_depth, 'b-', linewidth=2, label='Theoretical')

plt.xlabel('Disparity (pixels)')
plt.ylabel('Distance (meters)')
plt.title('Actual vs Theoretical Relationship\n(실제 vs 이론적 관계)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(0, 65)
plt.ylim(0, 5)

plt.tight_layout()
plt.show()

print("📊 시차-거리 관계 분석:")
print("   📐 이론적 관계: 역비례 곡선 (하이퍼볼라)")
print("   🔍 실제 데이터: 노이즈와 오차로 인해 분산됨")
print("   ⚠️ 시차가 작을수록 거리 측정 오차가 크게 증가")
print("   💡 가까운 물체일수록 더 정확한 거리 측정 가능")

## 🎯 실습 요약 및 결론

In [ ]:
print("🎉 스테레오비전 기초 실습 완료!")
print("\n📚 오늘 배운 내용:")
print("   1. 🖼️ 스테레오 이미지 쌍의 개념과 시차의 의미")
print("   2. 🔧 OpenCV StereoBM과 StereoSGBM 사용법")
print("   3. 🎛️ 매개변수가 결과에 미치는 영향")
print("   4. 🧮 시차 맵을 실제 거리로 변환하는 방법")
print("   5. 📊 시차-거리 관계의 이론과 실제")

print("\n🔑 핵심 공식:")
print(f"   💡 깊이 = (초점거리 × 베이스라인) / 시차")
print(f"   📐 depth = (f × b) / d")
print(f"   🔢 예시: ({focal_length} × {baseline}) / disparity")

print("\n🎯 실무 적용 팁:")
print("   ⚡ 속도가 중요하면 → StereoBM 사용")
print("   🎨 품질이 중요하면 → StereoSGBM 사용")
print("   📏 정확한 거리 측정을 위해서는 카메라 캘리브레이션 필수")
print("   🔧 매개변수 튜닝으로 환경에 최적화")

print("\n🚀 다음 단계:")
print("   1. 실제 스테레오 카메라 데이터로 실험")
print("   2. 카메라 캘리브레이션 학습")
print("   3. 3D 재구성 기법 탐구")
print("   4. 실시간 처리 최적화")

print("\n💪 수고하셨습니다! 다음 실습도 화이팅! 🎉")